In [1]:
import project as P

import os
from subprocess import call
from glob import glob

from tqdm import tqdm_notebook as tqdm

import xarray as xr

xr_open_ds = {'chunks' : {'time':1},
              'decode_coords' : False,
              'decode_times' : False}



droot = '/glade/p_old/decpred/CESM-DPLE_POPCICEhindcast'
case = 'g.e11_LENS.GECOIAF.T62_g16.009'
datestr = '024901-031612'
dple_root = '/glade/p_old/decpred/CESM-DPLE'

In [2]:
from dask.distributed import Client
from dask_jobqueue import PBSCluster

USER = os.environ['USER']

# Lots of arguments to this command are set in ~/.config/dask/jobqueue.yaml
cluster = PBSCluster(queue='regular',
                     cores = 36,
                     processes = 9,
                     memory = '100GB',                     
                     project = 'NCGD0033',
                     walltime = '04:00:00',
                     local_directory=f'/glade/scratch/{USER}/dask-tmp')
client = Client(cluster)

In [3]:
Nnodes = 4
cluster.scale(9*Nnodes)

In [4]:
clobber = True

v = 'PD'
files = sorted(glob(f'{dple_root}/monthly/PD/b.e11.BDP.f09_g16.????-??.???.pop.h.PD.*.nc'))

fname_out = lambda f: f.replace(dple_root,
                                f'{P.dirt}/CESM-DPLE').replace(f'.{v}.',
                                                               f'.{v}_200m.').replace(f'/{v}/',
                                                                                      f'/{v}_200m/')

file_out = fname_out(files[0])
if not os.path.exists(os.path.dirname(file_out)):
    call(['mkdir','-p',os.path.dirname(file_out)])

for f in tqdm(files):
    
    file_out = fname_out(f)

    if os.path.exists(file_out) and not clobber: 
        continue
        
    with xr.open_dataset(f,**xr_open_ds) as ds:
        ds = ds.sel(z_t=200e2,method='nearest')
        ds.reset_coords('z_t',inplace=True)
        ds.to_netcdf(file_out,unlimited_dims=['time'])

KeyboardInterrupt: 